In [1]:
import os
import re
import unsloth

os.environ["UNSLOTH_VLLM_STANDBY"] = "1"# [NEW] Extra 30% context lengths!

"""
unsloth                                  2025.11.6
unsloth_zoo                              2025.11.6
vllm                                     0.11.2
"""

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ng6309/datascience/anand/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


'\nunsloth                                  2025.11.6\nunsloth_zoo                              2025.11.6\nvllm                                     0.11.2\n'

In [2]:
from unsloth import FastLanguageModel
import torch

In [2]:
!pip list | grep unsloth 
!pip list | grep vllm 



unsloth                                  2025.11.6
unsloth_zoo                              2025.11.6
vllm                                     0.11.2


In [ ]:

max_seq_length = 8000 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "LiquidAI/LFM2-350M",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9, # Reduce if out of memory

    use_exact_model_name = True #for hugginface cache or repo mdoel name
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ng6309/datascience/anand/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
Switching to PyTorch attention since your Xformers is broken.

/home/ng6309/datascience/anand/vllm/lib/python3.10/site-packages/flash_attn_2_cuda.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZNK3c106SymInt6sym_neERKS0_
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Lfm2 patching. Transformers: 4.57.2. vLLM: 0.11.2.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.507 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.c

In [4]:
print(tokenizer.chat_template)

{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\n" if ns.system_prompt else "") + "List of tools: <|tool_list_start|>[" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
            {%- set tool = tool | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
        {%- if not loop.last -%}
            {%- set ns.system_prompt = ns.system_prompt + ", " -%}
        {%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "]<|tool_list_end|>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\n" + ns.system_prompt + "<|im_end|>\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\n" -}}
	{%- set content = mes

In [5]:
# # Clean GLM-style template with line-by-line tool format
# glm_chat_template = '''{{- bos_token -}}
# {%- set system_prompt = "" -%}
# {%- set ns = namespace(system_prompt="") -%}
# {%- if messages[0]["role"] == "system" -%}
# 	{%- set ns.system_prompt = messages[0]["content"] -%}
# 	{%- set messages = messages[1:] -%}
# {%- endif -%}
# {%- if tools -%}
# 	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
# 	{%- for tool in tools -%}
# 		{%- if tool is not string -%}
# 			{%- set tool = tool.function | tojson -%}
# 		{%- endif -%}
# 		{%- set ns.system_prompt = ns.system_prompt + tool -%}
# 		{%- if not loop.last -%}
# 			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
# 		{%- endif -%}
# 	{%- endfor -%}
# 	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
# {%- endif -%}
# {%- if ns.system_prompt -%}
# 	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
# {%- endif -%}
# {%- for message in messages -%}
# 	{{- "<|im_start|>" + message["role"] + "\\n" -}}
# 	{%- set content = message["content"] -%}
# 	{%- if content is not string -%}
# 		{%- set content = content | tojson -%}
# 	{%- endif -%}
# 	{%- if message["role"] == "tool" -%}
# 	{%- if message.get('name') -%}
#  		{%- set content = "<|observation|>\n<tool_name>" + message.get('name', 'unknown') + "</tool_name>\n<tool_response>" + content + "</tool_response>" -%}
# 	{%- else -%}
# 		{%- set content = "\n<|observation|>\n<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
# 	{%- endif -%}
# 	{%- endif -%}
# 	{{- content + "<|im_end|>\\n" -}}
# {%- endfor -%}
# {%- if add_generation_prompt -%}
# 	{{- "<|im_start|>assistant\\n" -}}
# {%- endif -%}'''

# tokenizer.chat_template = glm_chat_template

In [5]:
# Clean GLM-style template with line-by-line tool format
glm_chat_template = '''{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
			{%- set tool = tool.function | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
		{%- if not loop.last -%}
			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
		{%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\\n" -}}
	{%- set content = message["content"] -%}
	{%- if content is not string -%}
		{%- set content = content | tojson -%}
	{%- endif -%}
	{%- if message["role"] == "tool" -%}
	{%- if message.get('name') -%}
 		{%- set content = "<|observation|>\n<tool_name>" + message.get('name', 'unknown') + "</tool_name>\n<tool_response>" + content + "</tool_response>" -%}
	{%- else -%}
		{%- set content = "\n<|observation|>\n<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
	{%- endif -%}
	{%- endif -%}
	{{- content + "<|im_end|>\\n" -}}
{%- endfor -%}
{%- if add_generation_prompt -%}
	{{- "<|im_start|>assistant\\n" -}}
{%- endif -%}'''

tokenizer.chat_template = glm_chat_template

In [6]:
base_tools = {
  "web_search": {
    "type": "function",
    "function": {
      "name": "web_search",
      "description": "Search the web",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Search query"
          }
        },
        "required": [
          "query"
        ]
      },
      "return": {
        "type": "array",
        "items": {
          "type": "object"
        },
        "description": "The list of items having url,title,description."
      }
    }
  },
  "think": {
    "type": "function",
    "function": {
      "name": "think",
      "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.",
      "parameters": {
        "type": "object",
        "properties": {
          "thought": {
            "type": "string",
            "description": "A thought to think about."
          }
        },
        "required": [
          "thought"
        ]
      }
    }
  }
}

"""
  "response": {
    "type": "function",
    "function": {
      "name": "response",
      "description": "Send a message back to the user.",
      "parameters": {
        "type": "object",
        "properties": {
          "message": {
            "type": "string",
            "description": "The message to send to the user."
          }
        },
        "required": [
          "message"
        ]
      }
    }
  }
"""

'\n  "response": {\n    "type": "function",\n    "function": {\n      "name": "response",\n      "description": "Send a message back to the user.",\n      "parameters": {\n        "type": "object",\n        "properties": {\n          "message": {\n            "type": "string",\n            "description": "The message to send to the user."\n          }\n        },\n        "required": [\n          "message"\n        ]\n      }\n    }\n  }\n'

In [7]:
# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "name":"search_database","content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages,
    tools=list(base_tools.values()),
    tokenize=False, 
    add_generation_prompt=False
)

sp = re.split(r"<\|im_start\|>system\s*|\s*<\|im_end\|>", formatted, maxsplit=2)[1].strip()
print(sp)

# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool_call>{function-name}
<arg_key>{ar

In [8]:
# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "hiiii"}
]

formatted = tokenizer.apply_chat_template(
    messages, 
    tools=list(base_tools.values()), 
    tokenize=False, 
    add_generation_prompt=True
)

print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool

In [9]:
# Complete conversation with user, assistant, and tool messages (no think tags)
messages = [
    {"role": "user", "content": "What is the current status of candidate ID 12345?"},
    {"role": "assistant", "content": '<tool_call>get_candidate_status\n<arg_key>candidate_id</arg_key>\n<arg_value>12345</arg_value>\n</tool_call>'},
    {"role": "tool", "content": '{"candidate_id": "12345", "status": "Interview Scheduled", "position": "Clinical Research Associate", "date": "2023-11-20"}'},
    {"role": "assistant", "content": "The candidate with ID 12345 is currently in the \"Interview Scheduled\" stage for the position of Clinical Research Associate, with an interview date set for 2023-11-20."},
    {"role": "user", "content": "Can you also search for all candidates for the position of Data Scientist?"},
    {"role": "assistant", "content": '<tool_call>search_database\n<arg_key>query</arg_key>\n<arg_value>Data Scientist</arg_value>\n<arg_key>limit</arg_key>\n<arg_value>10</arg_value>\n</tool_call>'},
    {"role": "tool", "name":"search_database","content": '[{"candidate_id": "67890", "name": "John Doe", "status": "Applied"}, {"candidate_id": "67891", "name": "Jane Smith", "status": "Interview Completed"}]'},
    {"role": "assistant", "content": "I found 2 candidates for the Data Scientist position:\n1. John Doe (ID: 67890) - Status: Applied\n2. Jane Smith (ID: 67891) - Status: Interview Completed"}
]

formatted = tokenizer.apply_chat_template(
    messages,
    tools=list(base_tools.values()),
    tokenize=False, 
    add_generation_prompt=False
)

print(formatted)

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XML format:
<tool

## Pre fine-tuning for formatting

In [10]:
from datasets import load_dataset
dataset = load_dataset("Team-ACE/ToolACE")
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 11300
    })
})

In [11]:
import re

def convert_assistant_response_fixed(text: str) -> str:
    """
    Fixed version that handles multiple function calls properly.
    Converts format: [FunctionName(arg1="val1", arg2="val2")]
    To: <tool_call>FunctionName\n<arg_key>arg1</arg_key>\n<arg_value>val1</arg_value>...
    """
    text = text.strip()
    
    # Check if it looks like a tool call format
    if not text.startswith('[') or not text.endswith(']'):
        return text  # Not a tool call → return as-is
    
    # Remove outer brackets
    text = text[1:-1].strip()
    
    # Split multiple function calls by '),FunctionName(' or '),' patterns
    # This regex finds function boundaries
    function_pattern = r'(\w+(?:\s+\w+)*)\s*\((.*?)\)(?:,|$)'
    matches = re.finditer(function_pattern, text)
    
    results = []
    for match in matches:
        func_name = match.group(1).strip()
        args_str = match.group(2).strip()
        
        # Parse key=value pairs more carefully
        args = []
        # Use a more robust regex to handle quoted values with commas inside
        arg_pattern = r'(\w+)\s*=\s*(["\'])([^"\']*)\2'
        for arg_match in re.finditer(arg_pattern, args_str):
            key = arg_match.group(1)
            value = arg_match.group(3)  # The content between quotes
            args.append((key, value))
        
        # Build XML-style output for this function call
        xml_lines = [f"<tool_call>{func_name}"]
        for k, v in args:
            xml_lines.append(f"<arg_key>{k}</arg_key>")
            xml_lines.append(f"<arg_value>{v}</arg_value>")
        xml_lines.append("</tool_call>")
        
        results.append("\n".join(xml_lines))
    
    return "\n\n".join(results) if results else text

# Test cases
test_cases = [
    '[Get Zip Code Information(country="us", postal_code="10001")]',
    '[Get Zip Code Information(country="us", postal_code="10001"),Get Zip Code sdf(country="us", postal_code="10001")]',
    '[SEC Filings(identifier="AAPL")]',
]

print("Testing fixed function:\n")
for test in test_cases:
    print(f"Input: {test}")
    print(f"Output:\n{convert_assistant_response_fixed(test)}")
    print("-" * 80)

Testing fixed function:

Input: [Get Zip Code Information(country="us", postal_code="10001")]
Output:
<tool_call>Get Zip Code Information
<arg_key>country</arg_key>
<arg_value>us</arg_value>
<arg_key>postal_code</arg_key>
<arg_value>10001</arg_value>
</tool_call>
--------------------------------------------------------------------------------
Input: [Get Zip Code Information(country="us", postal_code="10001"),Get Zip Code sdf(country="us", postal_code="10001")]
Output:
<tool_call>Get Zip Code Information
<arg_key>country</arg_key>
<arg_value>us</arg_value>
<arg_key>postal_code</arg_key>
<arg_value>10001</arg_value>
</tool_call>

<tool_call>Get Zip Code sdf
<arg_key>country</arg_key>
<arg_value>us</arg_value>
<arg_key>postal_code</arg_key>
<arg_value>10001</arg_value>
</tool_call>
--------------------------------------------------------------------------------
Input: [SEC Filings(identifier="AAPL")]
Output:
<tool_call>SEC Filings
<arg_key>identifier</arg_key>
<arg_value>AAPL</arg_value>

In [12]:
import json

def prepare_tool(text):
    import json
    data = json.loads(text)
    tool_results = []
    for x in data:
        tool_results.append({"role":"tool","name":x['name'],"content":json.dumps((x['results']))})
    return tool_results

def convert_to_messages(example):
    _messages = []
    tools = example['system'].split("Here is a list of functions in JSON format that you can invoke:")[-1].strip().split("Should you decide to return the function call(s)")[0].replace(". \n","")
    for x in example['conversations']:
        if x['from'] =="assistant":
            _messages.append({"role":x['from'],"name":"","content":convert_assistant_response_fixed(x['value'])})
        elif x['from']=='tool':
            _messages.extend(prepare_tool(x['value']))
        else:
            _messages.append({"role":x['from'],"name":"","content":x['value']})

    return {"messages":_messages,"tools":tools}

dataset = dataset.map(convert_to_messages,num_proc=32)
dataset['train'][0]['messages']
# convert_to_messages(dataset['train'][0])

[{'content': "I'm considering investing and I'd like to know what's happening in the market right now. Could you get me the top market trends in the US?",
  'name': '',
  'role': 'user'},
 {'content': '<tool_call>Market Trends API\n<arg_key>trend_type</arg_key>\n<arg_value>MARKET_INDEXES</arg_value>\n<arg_key>country</arg_key>\n<arg_value>us</arg_value>\n</tool_call>',
  'name': '',
  'role': 'assistant'},
 {'content': '{"trends": [{"name": "S&P 500", "description": "Standard & Poor\'s 500 Index is a market-capitalization-weighted index of the 500 largest U.S. publicly traded companies.", "data": {"current_value": "4172.80", "percentage_change": "+0.68%"}}, {"name": "DOW J", "description": "Dow Jones Industrial Average is a price-weighted average of 30 blue-chip stocks that are generally the leaders in their industry.", "data": {"current_value": "34479.60", "percentage_change": "+0.47%"}}, {"name": "NASDAQ", "description": "The NASDAQ Composite is a broad-based capitalization-weighted 

In [13]:
def filter_tool(example):
    try:
            json.loads(example['tools'])
    except Exception as e:
            return False
    return True

dataset = dataset.filter(filter_tool,num_proc=32)
dataset

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations', 'messages', 'tools'],
        num_rows: 10782
    })
})

In [14]:
import json

def normalize_tool(tool):
    # If already in correct format, return as-is
    if "type" in tool and tool.get("type") == "function" and "function" in tool:
        return tool

    # Otherwise, assume it's the old flat format
    return {
        "type": "function",
        "function": {
            "name": tool["name"],
            "description": tool.get("description", ""),
            "parameters": {
                "type": "object",
                "properties": tool.get("parameters", {}).get("properties", {}),
                "required": tool.get("parameters", {}).get("required", [])
                # Note: ignore top-level "required": null
            }
        }
    }

def apply_chat_template(example):
    dynamic_tools = json.loads(example['tools'])
    normalized_dynamic_tools = [normalize_tool(t) for t in dynamic_tools]
    all_tools = list(base_tools.values()) + normalized_dynamic_tools

    prompt = tokenizer.apply_chat_template(
        example['messages'],
        tools=all_tools,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"prompt": prompt}

dataset = dataset.map(apply_chat_template,num_proc=32)
dataset


DatasetDict({
    train: Dataset({
        features: ['system', 'conversations', 'messages', 'tools', 'prompt'],
        num_rows: 10782
    })
})

In [15]:
print(dataset['train'][1]['messages'])

[{'content': 'Could you please find me some quotes about "inspiration"?', 'name': '', 'role': 'user'}, {'content': '<tool_call>Quotes by Keywords\n<arg_key>word</arg_key>\n<arg_value>inspiration</arg_value>\n</tool_call>', 'name': '', 'role': 'assistant'}, {'content': '{"quotes": [{"text": "The only way to achieve the impossible is to believe it is possible.", "author": "Charles Kingsleigh"}, {"text": "Don\'t watch the clock; do what it does. Keep going.", "author": "Sam Levenson"}, {"text": "Success is not the key to happiness. Happiness is the key to success. If you love what you are doing, you will be successful.", "author": "Albert Schweitzer"}]}', 'name': 'Quotes by Keywords', 'role': 'tool'}, {'content': 'Here are some inspiration quotes for you:\n\n1. "The only way to achieve the impossible is to believe it is possible." - Charles Kingsleigh\n2. "Don\'t watch the clock; do what it does. Keep going." - Sam Levenson\n3. "Success is not the key to happiness. Happiness is the key to

In [16]:
print(dataset['train'][1]['prompt'])

<|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
{"name": "Quotes by Keywords", "description": "Returns a list of quotes containing the specified keyword.", "p

In [18]:
dataset['train'][0]['tools']

'[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password for the new Ethereum address", "type": "string"}}, "required": ["password"]}, "required": null}, {"name": "Market Trends API", "description": "Get the latest market trends and relevant news for a specified country and language.", "parameters": {"type": "dict", "properties": {"trend_type": {"description": "Trend type.", "type": "string", "enum": ["MARKET_INDEXES", "MOST_ACTIVE", "GAINERS", "LOSERS", "CRYPTO", "CURRENCIES", "CLIMATE_LEADERS"]}, "country": {"description": "The country for which to get trends, specified as a 2-letter country code - see ISO 3166.", "type": "string", "default": "us"}, "language": {"description": "The language to use for the results, specified as a 2-letter language code - see

In [19]:
dataset['train'][0].keys()

dict_keys(['system', 'conversations', 'messages', 'tools', 'prompt'])

## Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [17]:
from trl import SFTTrainer, SFTConfig

train_dataset_sft = dataset['train'].select(range(250))
eval_dataset_sft = dataset['train'].select(range(250,750))

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset_sft,
    eval_dataset=eval_dataset_sft,
    args = SFTConfig(
        dataset_text_field = "prompt",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,# Increase to 4 for smoother training
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        # optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,

        report_to = "tensorboard", # Use this for WandB etc
        max_steps=100,

        # eval_strategy="steps",  # evaluate during training
        # eval_steps=10,                # evaluate every 10 steps
        # eval_delay=10,                # wait until step 10 to start evaluating
        # save_strategy="steps",        # optional: save model at eval points
        # save_steps=10,                # optional: save every eval

    ),
)
trainer.train()

#type(model) peft.peft_model.PeftModelForCausalLM
model.save_pretrained("lfm2-sft-tool")


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 250 | Num Epochs = 7 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 983,040 of 355,467,008 (0.28% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,4.178700
10,3.895800
15,3.700600
20,3.573900
25,3.243300
30,2.967200
35,2.887500
40,2.693800
45,2.497300
50,2.573000


In [19]:
# merge peft and save
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "LiquidAI/LFM2-350M",
    dtype=torch.bfloat16,
    device_map="auto",
)


# load adapter on top
peft_lfm2_sft_tool_model = PeftModel.from_pretrained(
    base_model,
    "lfm2-sft-tool",
)
peft_lfm2_sft_tool_model = peft_lfm2_sft_tool_model.merge_and_unload()

for param in peft_lfm2_sft_tool_model.parameters():
    param.requires_grad=True


In [22]:
# # merge peft and save
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel

# # load base model
# base_model = AutoModelForCausalLM.from_pretrained(
#     "LiquidAI/LFM2-350M",
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )

# # load adapter on top
# model = PeftModel.from_pretrained(
#     base_model,
#     "lfm2-sft-tool",
# )

# # merge
# model = model.merge_and_unload()

# # save merged model
# output_dir = "lfm2-sft-tool-merged"
# model.save_pretrained(output_dir)

# # save tokenizer
# tokenizer = AutoTokenizer.from_pretrained("LiquidAI/LFM2-350M")
# tokenizer.save_pretrained(output_dir)

# print(f"merged model saved to {output_dir}")

In [ ]:
# from trl import SFTTrainer, SFTConfig

# train_dataset_sft = dataset['train'].select(range(250))
# eval_dataset_sft = dataset['train'].select(range(250,750))

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = train_dataset_sft,
#     eval_dataset=eval_dataset_sft,
#     args = SFTConfig(
#         dataset_text_field = "prompt",
#         per_device_train_batch_size = 4,
#         gradient_accumulation_steps = 4,# Increase to 4 for smoother training
#         warmup_steps = 5,
#         num_train_epochs = 2, # Set this for 1 full training run.
#         learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
#         logging_steps = 5,
#         # optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,

#         report_to = "tensorboard", # Use this for WandB etc
#         max_steps=100,

#         # eval_strategy="steps",  # evaluate during training
#         # eval_steps=10,                # evaluate every 10 steps
#         # eval_delay=10,                # wait until step 10 to start evaluating
#         # save_strategy="steps",        # optional: save model at eval points
#         # save_steps=10,                # optional: save every eval

#     ),
# )
# trainer.train()

# #type(model) peft.peft_model.PeftModelForCausalLM``
# # model.save_pretrained("lfm2-sft-tool")


In [20]:
text = tokenizer.apply_chat_template(
    [{"role":"user","content":"search for when is modi born? and who won ipl match"}],
    tools = list(base_tools.values()),
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = peft_lfm2_sft_tool_model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.3,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|startoftext|><|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XM

In [21]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("LiquidAI/LFM2-350M")
sft_peft_model = PeftModel.from_pretrained(base_model, "lfm2-sft-tool").to('cuda')
text = tokenizer.apply_chat_template(
    [{"role":"user","content":"search for when is modi born? and who won ipl match"}],
    tools = list(base_tools.values()),
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = sft_peft_model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.3,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|startoftext|><|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XM

In [22]:
torch.cuda.empty_cache()
import gc
gc.collect()

432

In [23]:
def get_system_prompt_based_messages(example):
    sp = re.split(r"<\|im_start\|>system\s*|\s*<\|im_end\|>", example['prompt'], maxsplit=2)[1].strip()
    return {"prompt":[{"role":"system",'content':sp},*example['messages']]}

grpo_dataset = dataset.map(get_system_prompt_based_messages,num_proc=32, remove_columns=["messages","system", 'conversations', 'tools'])
grpo_dataset


DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 10782
    })
})

In [24]:
# def get_answer(exp): return {"answer":"22","samples":[12]}
# grpo_dataset = grpo_dataset.map(get_answer,num_proc=32)
# grpo_dataset
grpo_dataset.save_to_disk('tool-ace-dataset-grpo',num_proc=32)


Saving the dataset (32/32 shards): 100%|██████████| 10782/10782 [00:00<00:00, 11828.63 examples/s]


In [25]:
grpo_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 10782
    })
})

In [27]:
grpo_dataset['train'][0]['prompt']

[{'content': '# Tools\nYou may call one or more functions to assist with the user query.\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}\n{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}\n{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lo

In [28]:
from datasets import load_dataset, Dataset,DatasetDict
import json

def extract_training_examples(conversation):
    """
    Extract training examples from a conversation.

    For each assistant response that contains tool calls, create a training example with:
    - messages: all messages up to and including the user query
    - expected_tool_call: the assistant's tool call response
    - expected_tool_response: the corresponding tool response (if available)
    """
    messages = conversation['prompt']
    training_examples = []

    i = 0
    while i < len(messages):
        msg = messages[i]

        # Look for assistant messages with tool calls
        if msg.get('role') == 'assistant' and '<tool_call>' in msg.get('content', ''):
            # Collect all messages before this assistant response
            input_messages = messages[:i]

            # Only create example if there's at least one user message
            if any(m.get('role') == 'user' for m in input_messages):
                expected_tool_call = msg['content']

                # Look for the next tool response (if it exists)
                expected_tool_response = None
                if i + 1 < len(messages) and messages[i + 1].get('role') == 'tool':
                    expected_tool_response = messages[i + 1].get('content', '')

                training_examples.append({
                    'prompt': input_messages,
                    'expected_tool_call': expected_tool_call,
                    'expected_tool_response': expected_tool_response if expected_tool_response else ""
                })

        i += 1

    return training_examples


def transform_dataset(dataset):
    """
    Transform the GRPO dataset into a format with expected tool calls and responses.
    """
    all_examples = []

    for idx, example in enumerate(dataset):
        training_examples = extract_training_examples(example)
        all_examples.extend(training_examples)

        if idx % 1000 == 0:
            print(f"Processed {idx}/{len(dataset)} conversations, generated {len(all_examples)} training examples so far...")

    return all_examples


# Load the original dataset
print("Loading dataset...")
tool_ace_ds_grpo = load_dataset('tool-ace-dataset-grpo', num_proc=32)

print(f"Original dataset size: {len(tool_ace_ds_grpo['train'])} conversations")
print()

# Transform the train split
print("Transforming dataset...")
transformed_examples = transform_dataset(tool_ace_ds_grpo['train'])

print(f"\nGenerated {len(transformed_examples)} training examples from {len(tool_ace_ds_grpo['train'])} conversations")
print()

# Show first 3 examples
print("="*80)
print("SAMPLE TRANSFORMED EXAMPLES:")
print("="*80)

for i in range(min(3, len(transformed_examples))):
    example = transformed_examples[i]
    print(f"\nExample {i+1}:")
    print("-" * 80)
    print(f"Number of input messages: {len(example['prompt'])}")
    print()

    # Show last 2 messages (typically system + user, or previous context + user)
    print("Last 2 input messages:")
    for msg in example['prompt'][-2:]:
        role = msg.get('role', 'NO_ROLE')
        content = msg.get('content', 'NO_CONTENT')
        print(f"\n  Role: {role}")
        print(f"  Content: {content[:300]}{'...' if len(content) > 300 else ''}")

    print("\n" + "-" * 80)
    print("Expected tool call:")
    print(example['expected_tool_call'][:500] + ('...' if len(example['expected_tool_call']) > 500 else ''))

    print("\n" + "-" * 80)
    print("Expected tool response:")
    print(example['expected_tool_response'][:500] + ('...' if len(example['expected_tool_response']) > 500 else ''))
    print("\n" + "="*80)

# Create a new dataset
print("\nCreating new DatasetDict...")
train_dataset = Dataset.from_list(transformed_examples)
new_dataset = DatasetDict({
    'train': train_dataset
})
print(new_dataset)
# Save the dataset
output_path = 'tool-ace-dataset-grpo-with-expected'
print(f"\nSaving transformed dataset to '{output_path}'...")
new_dataset.save_to_disk(output_path)

print(f"\n✓ Dataset saved successfully!")
print(f"  - Original conversations: {len(dataset['train'])}")
print(f"  - Training examples generated: {len(transformed_examples)}")
print(f"  - Saved to: {output_path}")


Loading dataset...


Generating train split: 10782 examples [00:00, 11648.76 examples/s]


Original dataset size: 10782 conversations

Transforming dataset...
Processed 0/10782 conversations, generated 2 training examples so far...
Processed 1000/10782 conversations, generated 1696 training examples so far...
Processed 2000/10782 conversations, generated 2549 training examples so far...
Processed 3000/10782 conversations, generated 3386 training examples so far...
Processed 4000/10782 conversations, generated 4239 training examples so far...
Processed 5000/10782 conversations, generated 5092 training examples so far...
Processed 6000/10782 conversations, generated 5931 training examples so far...
Processed 7000/10782 conversations, generated 6756 training examples so far...
Processed 8000/10782 conversations, generated 7595 training examples so far...
Processed 9000/10782 conversations, generated 8448 training examples so far...
Processed 10000/10782 conversations, generated 9312 training examples so far...

Generated 9962 training examples from 10782 conversations

SAMPLE T

Saving the dataset (1/1 shards): 100%|██████████| 9962/9962 [00:00<00:00, 702967.02 examples/s]


✓ Dataset saved successfully!
  - Original conversations: 10782
  - Training examples generated: 9962
  - Saved to: tool-ace-dataset-grpo-with-expected


## GRPO Training

In [ ]:
import re
from datasets import load_dataset,load_from_disk
from unsloth import FastLanguageModel

# Load model using Unsloth's FastLanguageModel
max_seq_length = 8000

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lfm2-sft-tool",  # Load the SFT fine-tuned model directly
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = False,
    gpu_memory_utilization = 0.9,
)

dataset = load_from_disk('tool-ace-dataset-grpo-with-expected')
print(dataset)

glm_chat_template = '''{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
			{%- set tool = tool.function | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
		{%- if not loop.last -%}
			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
		{%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\\n" -}}
	{%- set content = message["content"] -%}
	{%- if content is not string -%}
		{%- set content = content | tojson -%}
	{%- endif -%}
	{%- if message["role"] == "tool" -%}
	{%- if message.get('name') -%}
 		{%- set content = "<|observation|>\n<tool_name>" + message.get('name', 'unknown') + "</tool_name>\n<tool_response>" + content + "</tool_response>" -%}
	{%- else -%}
		{%- set content = "\n<|observation|>\n<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
	{%- endif -%}
	{%- endif -%}
	{{- content + "<|im_end|>\\n" -}}
{%- endfor -%}
{%- if add_generation_prompt -%}
	{{- "<|im_start|>assistant\\n" -}}
{%- endif -%}'''
base_tools = {
  "web_search": {
    "type": "function",
    "function": {
      "name": "web_search",
      "description": "Search the web",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Search query"
          }
        },
        "required": [
          "query"
        ]
      },
      "return": {
        "type": "array",
        "items": {
          "type": "object"
        },
        "description": "The list of items having url,title,description."
      }
    }
  },
  "think": {
    "type": "function",
    "function": {
      "name": "think",
      "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.",
      "parameters": {
        "type": "object",
        "properties": {
          "thought": {
            "type": "string",
            "description": "A thought to think about."
          }
        },
        "required": [
          "thought"
        ]
      }
    }
  }
}

tokenizer.chat_template = glm_chat_template



==((====))==  Unsloth 2025.12.5: Fast Lfm2 patching. Transformers: 4.57.3. vLLM: 0.11.2.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.507 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
DatasetDict({
    train: Dataset({
        features: ['prompt', 'expected_tool_call', 'expected_tool_response'],
        num_rows: 9962
    })
})


In [4]:
from transformers import TextStreamer
text = tokenizer.apply_chat_template(
    [{"role":"user","content":"search for when is modi born? and who won ipl match"}],
    tools = list(base_tools.values()),
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.3,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|startoftext|><|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XM

<tool_call>{web_search}
<query>When is Modi born? Who won IPL match?</query>
</tool_call><|im_end|>


In [7]:
maximum_length = 6000
max_prompt_length = maximum_length + 1 # + 1 just in case!

max_completion_length  = 11

def detect_leading_tool_call(prompts,completions,**kwargs):
    scores = []
    for cp in completions:
        if "<tool_call>" ==cp[0]['content'][:11]:
            scores.append(1.0)
        elif "<tool_call>" in cp[0]['content']:
            scores.append(0.5)
        else:
            scores.append(0.0)
    return scores




from unsloth import vLLMSamplingParams
vllm_sampling_params = vLLMSamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer

# First create the config without vllm_sampling_params
training_args = GRPOConfig(
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    # optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 300,
    # save_steps = 100,
    report_to = "tensorboard", # Can use Weights & Biases
    output_dir = "outputs",
    torch_compile = False,  # Disable torch.compile to avoid dimension mismatch errors

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

# Add vllm_sampling_params after creation to avoid JSON serialization issues
training_args.vllm_sampling_params = vllm_sampling_params

# Patch the to_dict method to exclude vllm_sampling_params from serialization
original_to_dict = training_args.to_dict
def patched_to_dict():
    d = original_to_dict()
    # Remove vllm_sampling_params from the dict to avoid JSON serialization error
    d.pop('vllm_sampling_params', None)
    return d
training_args.to_dict = patched_to_dict

# For optional training + evaluation
train_dataset_for_grpo = dataset['train'].select(range(250,300))
# new_dataset = dataset.train_test_split(test_size = 0.01)

tokenizer.chat_template = glm_chat_template

def train(reward_funcs):
    trainer = GRPOTrainer(
        model = model,
        processing_class = tokenizer,
        reward_funcs = reward_funcs,
        args = training_args,
        train_dataset = train_dataset_for_grpo,

        # For optional training + evaluation
        # train_dataset = new_dataset["train"],
        # eval_dataset = new_dataset["test"],
    )
    trainer.train()

train([detect_leading_tool_call])
    
# model.save_pretrained("lfm2-sft-tool-grpo")

print("Type of model:", type(model))

The model is already on multiple devices. Skipping the move to device specified in `args`.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 12 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 983,040 of 355,467,008 (0.28% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / check_tool_call_token_answer / mean,rewards / check_tool_call_token_answer / std
1,0.000900,1.000000,0.000000,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0.910441,1.000000,0.000000
2,0.000500,0.875000,0.231455,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.491404,0.875000,0.341565
3,0.000400,0.812500,0.408232,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.414102,0.812500,0.403113
4,0.000400,0.687500,0.258775,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.360436,0.687500,0.478714
5,0.000600,0.968750,0.088388,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.604961,0.968750,0.125000
6,0.000900,0.875000,0.231455,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.865029,0.875000,0.341565
7,0.000700,1.000000,0.000000,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.696824,1.000000,0.000000
8,0.000800,1.000000,0.000000,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.792080,1.000000,0.000000
9,0.001000,0.937500,0.176777,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,1.034207,0.937500,0.250000
10,0.000500,1.000000,0.000000,11.000000,11.000000,11.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.466009,1.000000,0.000000


KeyboardInterrupt: 

In [5]:
from transformers import TextStreamer
tokenizer.chat_template = glm_chat_template

text = tokenizer.apply_chat_template(
    [{"role":"user","content":"when is modi born?"}],
    tools = list(base_tools.values()),
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.3,
    max_new_tokens = 2048,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|startoftext|><|startoftext|><|im_start|>system
# Tools
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}
{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}
</tools>
For each function call, output the function name and arguments within the following XM

"web_search", "arguments": {"query": "Modi birth date"}}
</tool_call>
{"arg_key": "birth_date", "arg_value": "1984-06-14"}
</tool_call><|im_end|>


In [52]:
def generate(messages,**kwargs):
    text = tokenizer.apply_chat_template(
        messages,
        tools = list(base_tools.values()),
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
    )
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(**tokenizer(text, return_tensors = "pt").to("cuda"),**kwargs)
    
    generated_ids = outputs[0][inputs['input_ids'].shape[1]:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=False)
    return generated_text

In [53]:
dataset['train'][0]['prompt']

[{'content': '# Tools\nYou may call one or more functions to assist with the user query.\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}\n{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}\n{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lo

In [57]:
generate(dataset['train'][0]['prompt'])

'<tool_call>{web_search, trend_type="MARKET_INDEXES"}}\n<arg_key>get_US_market_trends</arg_key>\n<arg_value>{get_trends("US", "MARKET_INDEXES")}</arg_value>\n</tool_call><|im_end|>'

In [58]:
print(dataset['train'][0]['prompt'])
print(dataset['train'][0]["expected_tool_call"][:11])
print(dataset['train'][0]["expected_tool_response"])


[{'content': '# Tools\nYou may call one or more functions to assist with the user query.\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "web_search", "description": "Search the web", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}, "return": {"type": "array", "items": {"type": "object"}, "description": "The list of items having url,title,description."}}\n{"name": "think", "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.", "parameters": {"type": "object", "properties": {"thought": {"type": "string", "description": "A thought to think about."}}, "required": ["thought"]}}\n{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lo

In [59]:
from transformers import TextStreamer
tokenizer.chat_template = glm_chat_template


scores = []
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import json

def get_first_token_tool_score(example,res):
    return {"score":res[:11]==example['expected_tool_call'][:11]}

for example in tqdm(dataset['train'].select(range(100))):
    res = generate(example['prompt'],temperature = 0.3,max_new_tokens = 11)
    scores.append(get_first_token_tool_score(example,res))
    
df = pd.DataFrame(scores)
res = df['score'].astype(int).describe()
Path('scores.json').write_text(json.dumps(res.to_dict(),indent=2))

100%|██████████| 100/100 [00:10<00:00,  9.74it/s]


135

In [60]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [ ]:
from vllm import LLM, SamplingParams
from datasets import load_from_disk
from transformers import AutoTokenizer
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm
# Clean GLM-style template with line-by-line tool format
glm_chat_template = '''{{- bos_token -}}
{%- set system_prompt = "" -%}
{%- set ns = namespace(system_prompt="") -%}
{%- if messages[0]["role"] == "system" -%}
	{%- set ns.system_prompt = messages[0]["content"] -%}
	{%- set messages = messages[1:] -%}
{%- endif -%}
{%- if tools -%}
	{%- set ns.system_prompt = ns.system_prompt + ("\\n" if ns.system_prompt else "") + "# Tools\\nYou may call one or more functions to assist with the user query.\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\\n" -%}
	{%- for tool in tools -%}
		{%- if tool is not string -%}
			{%- set tool = tool.function | tojson -%}
		{%- endif -%}
		{%- set ns.system_prompt = ns.system_prompt + tool -%}
		{%- if not loop.last -%}
			{%- set ns.system_prompt = ns.system_prompt + "\\n" -%}
		{%- endif -%}
	{%- endfor -%}
	{%- set ns.system_prompt = ns.system_prompt + "\\n</tools>\\nFor each function call, output the function name and arguments within the following XML format:\\n<tool_call>{function-name}\\n<arg_key>{arg_key}</arg_key>\\n<arg_value>{arg_value}</arg_value>\\n...\\n</tool_call>" -%}
{%- endif -%}
{%- if ns.system_prompt -%}
	{{- "<|im_start|>system\\n" + ns.system_prompt + "<|im_end|>\\n" -}}
{%- endif -%}
{%- for message in messages -%}
	{{- "<|im_start|>" + message["role"] + "\\n" -}}
	{%- set content = message["content"] -%}
	{%- if content is not string -%}
		{%- set content = content | tojson -%}
	{%- endif -%}
	{%- if message["role"] == "tool" -%}
	{%- if message.get('name') -%}
 		{%- set content = "<|observation|>\n<tool_name>" + message.get('name', 'unknown') + "</tool_name>\n<tool_response>" + content + "</tool_response>" -%}
	{%- else -%}
		{%- set content = "\n<|observation|>\n<|tool_response_start|>" + content + "<|tool_response_end|>" -%}
	{%- endif -%}
	{%- endif -%}
	{{- content + "<|im_end|>\\n" -}}
{%- endfor -%}
{%- if add_generation_prompt -%}
	{{- "<|im_start|>assistant\\n" -}}
{%- endif -%}'''


base_tools = {
  "web_search": {
    "type": "function",
    "function": {
      "name": "web_search",
      "description": "Search the web",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Search query"
          }
        },
        "required": [
          "query"
        ]
      },
      "return": {
        "type": "array",
        "items": {
          "type": "object"
        },
        "description": "The list of items having url,title,description."
      }
    }
  },
  "think": {
    "type": "function",
    "function": {
      "name": "think",
      "description": "Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.",
      "parameters": {
        "type": "object",
        "properties": {
          "thought": {
            "type": "string",
            "description": "A thought to think about."
          }
        },
        "required": [
          "thought"
        ]
      }
    }
  }
}


llm = LLM(
    model="lfm2-sft-tool-merged-grpo",
    tensor_parallel_size=1,
    gpu_memory_utilization=0.45,
    # max_model_len=2048
)

tokenizer = AutoTokenizer.from_pretrained("LiquidAI/LFM2-350M")
tokenizer.chat_template = glm_chat_template

# sampling params
sampling_params = SamplingParams(
    temperature=0.3,
    max_tokens=11,
)

# Test with a simple prompt
messages = [
    {"role": "user", "content": "Hi"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Prompt:", prompt)
print("\nGenerating response...")

outputs = llm.generate(prompt, sampling_params)

for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated: {generated_text}")


INFO 12-03 15:58:54 [utils.py:253] non-default args: {'gpu_memory_utilization': 0.45, 'disable_log_stats': True, 'model': 'lfm2-sft-tool-merged-grpo'}
INFO 12-03 15:58:59 [model.py:631] Resolved architecture: Lfm2ForCausalLM
INFO 12-03 15:58:59 [model.py:1745] Using max model len 128000


2025-12-03 15:58:59,321	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-03 15:58:59 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 12-03 15:58:59 [config.py:308] Disabling cascade attention since it is not supported for hybrid models.
INFO 12-03 15:58:59 [config.py:432] Setting attention block size to 16 tokens to ensure that attention page size is >= mamba page size.
INFO 12-03 15:58:59 [config.py:456] Padding mamba page size by 700.00% to ensure that mamba page size and attention page size are exactly equal.
WARNING 12-03 15:58:59 [system_utils.py:103] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=514017) INFO 12-03 15:59:02 [core.py:93] Initializing a V1 LLM engine (v0.11.2) with config: model='lfm2-sft-tool-merged-grpo', speculative_config=None, tokenizer='lfm2-sft-tool-merged-grpo', skip_tokeni

(EngineCore_DP0 pid=514017) [2025-12-03 15:59:03] INFO _optional_torch_c_dlpack.py:119: JIT-compiling torch-c-dlpack-ext to cache...


In [ ]:
llm.generate(['hi','what'],SamplingParams(max_tokens=10))

In [ ]:


# load dataset
dataset = load_from_disk('tool-ace-dataset-grpo-with-expected')
dataset = dataset['train']

def prepare(example):
    return {"vllm_prompt":tokenizer.apply_chat_template(
    example['prompt'],tools=list(base_tools.values()),tokenize=False,add_generation_prompt=True
    )}

dataset = dataset.map(prepare,num_proc=32)
sampling_params = SamplingParams(
    temperature=0.3,
    max_tokens=11,
)

# generate with vllm - batched automatically
outputs = llm.generate(list(dataset['vllm_prompt']), sampling_params)

completions = [x.outputs[0].text for x in outputs]

dataset = dataset.add_column('completions', completions)

# save results
df = dataset.select_columns(['completions','expected_tool_call']).to_pandas()
df
df['score'] = df.apply(lambda row: int(row['completions'][:11] == row['expected_tool_call'][:11]), axis=1)
res = df['score'].describe()
Path('scores-grpo.json').write_text(json.dumps(res.to_dict(), indent=2))

print(f"accuracy: {df['score'].mean():.3f}")